# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Boston Housing Data once more

We pre-processed the Boston Housing data the same way we did before:

- We dropped `'ZN'` and `'NOX'` completely
- We categorized `'RAD'` in 3 bins and `'TAX'` in 4 bins
- We transformed `'RAD'` and `'TAX'` to dummy variables and dropped the first variable
- We used min-max-scaling on `'B'`, `'CRIM'`, and `'DIS'` (and logtransformed all of them first, except `'B'`)
- We used standardization on `'AGE'`, `'INDUS'`, `'LSTAT'`, and `'PTRATIO'` (and logtransformed all of them first, except for `'AGE'`) 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(['NOX', 'ZN'],axis=1)

# First, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# First, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix='TAX', drop_first=True)
rad_dummy = pd.get_dummies(bins_rad, prefix='RAD', drop_first=True)
boston_features = boston_features.drop(['RAD', 'TAX'], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

age = boston_features['AGE']
b = boston_features['B']
logcrim = np.log(boston_features['CRIM'])
logdis = np.log(boston_features['DIS'])
logindus = np.log(boston_features['INDUS'])
loglstat = np.log(boston_features['LSTAT'])
logptratio = np.log(boston_features['PTRATIO'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['CRIM'] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['AGE'] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features['INDUS'] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features['PTRATIO'] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(6, 24]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,0,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,0,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,0,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,0,0,0


In [4]:
boston_target = pd.DataFrame(data=boston['target'])
boston_target.columns = ['MEDV']

In [5]:
boston_target

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2
...,...
501,22.4
502,20.6
503,23.9
504,22.0


In [6]:
list(boston_features.columns.values)

['CRIM',
 'INDUS',
 'CHAS',
 'RM',
 'AGE',
 'DIS',
 'PTRATIO',
 'B',
 'LSTAT',
 'RAD_(6, 24]',
 'TAX_(270, 360]',
 'TAX_(360, 712]']

## Perform stepwise selection

The function for stepwise selection is copied below. Use this function provided on your preprocessed Boston Housing data.

In [8]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [11]:
# Your code here
included = stepwise_selection(boston_features, boston_target, initial_list=list(boston_features.columns.values), threshold_in=0.01, threshold_out = 0.05, verbose=True)

/opt/conda/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


Drop AGE                            with p-value 0.821467
Drop CRIM                           with p-value 0.366451
Drop RAD_(6, 24]                    with p-value 0.0699778


In [12]:
included

['INDUS',
 'CHAS',
 'RM',
 'DIS',
 'PTRATIO',
 'B',
 'LSTAT',
 'TAX_(270, 360]',
 'TAX_(360, 712]']

### Build the final model again in Statsmodels

In [14]:
# Your code here
boston_dropped = boston_features
boston_dropped.drop('AGE', axis=1, inplace=True)
boston_dropped.drop('CRIM', axis=1, inplace=True)
boston_dropped.drop('Drop RAD_(6, 24]', axis=1, inplace=True)

KeyError: "['Drop RAD_(6, 24]'] not found in axis"

In [15]:
boston_dropped

,INDUS,CHAS,RM,DIS,PTRATIO,B,LSTAT,"RAD_(6, 24]","TAX_(270, 360]","TAX_(360, 712]"
0,-1.704344,0.0,6.575,0.542096,-1.443977,1.000000,-1.275260,0,1,0
1,-0.263239,0.0,6.421,0.623954,-0.230278,1.000000,-0.263711,0,0,0
2,-0.263239,0.0,7.185,0.623954,-0.230278,0.989737,-1.627858,0,0,0
3,-1.778965,0.0,6.998,0.707895,0.165279,0.994276,-2.153192,0,0,0
4,-1.778965,0.0,7.147,0.707895,0.165279,1.000000,-1.162114,0,0,0
...,...,...,...,...,...,...,...,...,...,...
501,0.410792,0.0,6.593,0.331081,1.095518,0.987619,-0.169811,0,1,0
502,0.410792,0.0,6.120,0.297277,1.095518,1.000000,-0.274682,0,1,0
503,0.410792,0.0,6.976,0.274575,1.095518,1.000000,-1.067939,0,1,0
504,0.410792,0.0,6.794,0.315551,1.095518,0.991301,-0.836660,0,1,0


In [16]:
list(boston_dropped.columns.values)

['INDUS',
 'CHAS',
 'RM',
 'DIS',
 'PTRATIO',
 'B',
 'LSTAT',
 'RAD_(6, 24]',
 'TAX_(270, 360]',
 'TAX_(360, 712]']

In [19]:
boston_dropped.drop(boston_dropped.columns[7], axis=1, inplace=True)

The stepwise procedure mentions that `'INDUS'` was added with a p-value of 0.0017767, but our statsmodels output returns a p-value of 0.000. Use some of the stepwise procedure logic to find the intuition behind this!

In [20]:
boston_dropped

,INDUS,CHAS,RM,DIS,PTRATIO,B,LSTAT,"TAX_(270, 360]","TAX_(360, 712]"
0,-1.704344,0.0,6.575,0.542096,-1.443977,1.000000,-1.275260,1,0
1,-0.263239,0.0,6.421,0.623954,-0.230278,1.000000,-0.263711,0,0
2,-0.263239,0.0,7.185,0.623954,-0.230278,0.989737,-1.627858,0,0
3,-1.778965,0.0,6.998,0.707895,0.165279,0.994276,-2.153192,0,0
4,-1.778965,0.0,7.147,0.707895,0.165279,1.000000,-1.162114,0,0
...,...,...,...,...,...,...,...,...,...
501,0.410792,0.0,6.593,0.331081,1.095518,0.987619,-0.169811,1,0
502,0.410792,0.0,6.120,0.297277,1.095518,1.000000,-0.274682,1,0
503,0.410792,0.0,6.976,0.274575,1.095518,1.000000,-1.067939,1,0
504,0.410792,0.0,6.794,0.315551,1.095518,0.991301,-0.836660,1,0


## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [21]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression


In [22]:
# Your code here

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=5)
selector = selector.fit(boston_features, boston_target)

/opt/conda/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fit the linear regression model again using the 5 selected columns

In [23]:
selector

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                               normalize=False),
    n_features_to_select=5, step=1, verbose=0)

In [24]:
selector.support_

array([False,  True,  True,  True, False,  True,  True, False, False])

In [27]:
selector.ranking_

array([5, 1, 1, 1, 4, 1, 1, 3, 2])

In [30]:
# Your code here
estimators = selector.estimator_
print(estimators.coef_)
print(estimators.intercept_)

[ 2.93498961  3.43718997 -6.58036332  4.65357304 -6.25217488]
-0.49739821537886897


Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [31]:
type(selector)

sklearn.feature_selection.rfe.RFE

In [34]:
# Your code here
# From official documentation for RFE.predict(self, X)
# Reduce X to the selected features and then predict using the
selector.predict(boston_features)

array([31.16165902, 23.76929453, 34.8764376 , 36.98692436, 31.32930715,
       29.06938762, 18.83320099, 14.70915944,  8.01751688, 14.90750061,
       14.49961186, 17.84334784, 15.96671008, 23.35014491, 21.5475942 ,
       22.80420786, 25.67823103, 17.66801955, 17.36837517, 19.86812537,
       12.67589909, 18.42874828, 15.95977353, 14.09380242, 16.3420057 ,
       13.99038842, 16.58165948, 15.0909717 , 20.71269856, 22.06463506,
       11.91943166, 19.11352766,  9.2779531 , 14.33996426, 13.34927881,
       22.57757018, 20.30738333, 22.88337432, 21.8018023 , 31.92538034,
       41.52527107, 31.06370984, 27.07153202, 24.77046631, 21.62254369,
       20.00275986, 16.96128095, 14.4887785 ,  7.13719738, 14.42912675,
       17.2790188 , 21.48105026, 28.91702238, 22.28080957, 15.84257515,
       31.73722975, 26.72444484, 32.38609452, 24.47989229, 21.05648509,
       16.58337781, 16.34295586, 26.03847483, 23.20882414, 25.68084118,
       29.5199626 , 19.61369356, 22.40808622, 16.44091927, 21.58

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [ ]:
# Your code here

# r_squared is 0.742981  
# adjusted_r_squared is 0.740411

## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Boston Housing dataset! 